<a href="https://colab.research.google.com/github/Base-R-Best-R/Auction/blob/main/Code/Models/Colab/RF_Train_Sk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RF in Sklearn

In [ ]:
# Imports
import pandas as pd
import numpy as np
# !pip install pyreadr
import pyreadr
import sklearn

In [11]:
# Import RDS training Data
url = "https://github.com/Base-R-Best-R/Auction/blob/main/Data/Bid%20Tab%20RDS/Colab%20Transfer/csv/Aucs_df_feateng_train.csv"
dat_train = pd.read_csv(url)
dat_train.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 28: expected 1 fields, saw 412\nSkipping line 72: expected 1 fields, saw 2\nSkipping line 170: expected 1 fields, saw 3\nSkipping line 184: expected 1 fields, saw 6\nSkipping line 210: expected 1 fields, saw 4\nSkipping line 215: expected 1 fields, saw 4\nSkipping line 220: expected 1 fields, saw 4\nSkipping line 225: expected 1 fields, saw 4\nSkipping line 230: expected 1 fields, saw 4\nSkipping line 236: expected 1 fields, saw 4\nSkipping line 241: expected 1 fields, saw 4\nSkipping line 246: expected 1 fields, saw 4\nSkipping line 251: expected 1 fields, saw 4\nSkipping line 256: expected 1 fields, saw 4\nSkipping line 261: expected 1 fields, saw 4\nSkipping line 266: expected 1 fields, saw 4\nSkipping line 277: expected 1 fields, saw

,<!DOCTYPE html>
0,"<html lang=""en"" data-color-mode=""auto"" data-li..."
1,<head>
2,"<meta charset=""utf-8"">"
3,"<link rel=""dns-prefetch"" href=""https://githu..."
4,"<link rel=""dns-prefetch"" href=""https://avata..."
